In [7]:
import json, os, pickle, random
import numpy as np
from tqdm.notebook import tqdm
import pandas as pd

import torch
from torch.utils.data import Dataset, DataLoader, TensorDataset
from torch.optim.lr_scheduler import ReduceLROnPlateau
import torch.nn as nn

import qiskit
from qiskit import QuantumCircuit, execute
from qiskit.compiler import transpile
from qiskit_aer import AerSimulator, QasmSimulator
from qiskit.converters import circuit_to_dag, dag_to_circuit
from qiskit.quantum_info import SparsePauliOp, Operator
from qiskit.circuit.library import CXGate, RXGate, IGate, ZGate
from qiskit.providers.fake_provider import FakeMontreal, FakeLima,FakeGuadalupe,FakeSherbrooke,FakePrague,FakeCairo
from blackwater.data.utils import (
    generate_random_pauli_sum_op,
    create_estimator_meas_data,
    circuit_to_graph_data_json,
    get_backend_properties_v1,
    encode_pauli_sum_op,
    create_meas_data_from_estimators
)
from qiskit.circuit.random import random_circuit
from tqdm import tqdm_notebook
from mlp import MLP1, MLP2, MLP3, encode_data

from mbd_utils import cal_z_exp, generate_disorder, construct_mbl_circuit, calc_imbalance, modify_and_add_noise_to_model

import matplotlib.pyplot as plt
import seaborn as sns
from noise_utils import AddNoise, RemoveReadoutErrors

In [8]:
def fix_random_seed(seed=0):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # if you are using multi-GPU.
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True
    print(f'random seed fixed to {seed}')

In [9]:
with open('./multi_backend_model_data/tianyan_error.json', 'r') as f:
    data = json.load(f)
properties_dict = {'tianyan':data}

In [10]:
# 将HLD_list追加保存到json文件
def save_HLD(model_name,HLD_list):
    # 定义两个字典
    HLD_dict = {}
    HLD_dict[model_name] = HLD_list
    # 读取已有的 JSON 文件内容
    with open('./multi_backend_model_data/HLD.json', 'r') as file:
        data = json.load(file)

    # 将第二个字典追加到已有内容中
    data.update(HLD_dict)

    # 将更新后的内容写回 JSON 文件
    with open('./multi_backend_model_data/HLD.json', 'w') as file:
        json.dump(data, file)


# 将2个字典互补
def complement_dicts(dict1, dict2):
    # 找到两个字典的并集键集合
    all_keys = set(dict1.keys()) | set(dict2.keys())

    # 在每个字典中补足缺失的键值对
    for key in all_keys:
        dict1.setdefault(key, 0)
        dict2.setdefault(key, 0)

    return dict1, dict2


def chunk_list(input_list, chunk_size):
    return [input_list[i:i + chunk_size] for i in range(0, len(input_list), chunk_size)]

# 加载训练和测试数据文件
def load_circuits(data_files, f_ext='.pk'):
    backend_name_list = []
    circuits = []
    ideal_exp_vals = []
    noisy_exp_vals_1 = []
    noisy_exp_vals_2 = []
    noisy_exp_vals_3 = []
    for data_file in data_files:
        if f_ext == '.json':
            for entry in json.load(open(data_file, 'r')):
                # 获取当前线路对应的后端，这个后端名称可以从数据文件的名字获得
                data_backend_name = os.path.basename(data_file).split('.')[0].split('_')[0]
                backend_name_list.append(data_backend_name)
                circuits.append(QuantumCircuit.from_qasm_str(entry['circuit']))
                ideal_exp_vals.append(entry['ideal_exp_value'])
                noisy_exp_vals_1.append(entry['noisy_exp_values_1'])
                noisy_exp_vals_2.append(entry['noisy_exp_values_2'])
                noisy_exp_vals_3.append(entry['noisy_exp_values_3'])
        elif f_ext == '.pk':
            for entry in pickle.load(open(data_file, 'rb')):
                # 获取当前线路对应的后端，这个后端名称可以从数据文件的名字获得
                data_backend_name = os.path.basename(data_file).split('.')[0].split('_')[0]
                backend_name_list.append(data_backend_name)
                circuits.append(entry['circuit'])
                ideal_exp_vals.append(entry['ideal_exp_value'])
                noisy_exp_vals_1.append(entry['noisy_exp_values_1'])
                noisy_exp_vals_2.append(entry['noisy_exp_values_2'])
                noisy_exp_vals_3.append(entry['noisy_exp_values_3'])
    return circuits, ideal_exp_vals, noisy_exp_vals_1,noisy_exp_vals_2,noisy_exp_vals_3,backend_name_list


# 获取数组中前25%大的数据和第25%-50%大的数据
def get_25_50(arr):
    arr.sort()  # 对数组进行排序
    quarter = len(arr) // 4  # 计算数组长度的四分之一
    first_25_percent = arr[-quarter:]  # 获取数组中前25%大的数据
    second_25_to_50_percent = arr[-2*quarter:-quarter]  # 获取数组中第25%-50%大的数据
    return first_25_percent, second_25_to_50_percent

# 将一个形状如tensor([0.2600, 0.0900])的数据按照第一个为state，第二个位qubits转换为对应的二进制数
def get_state(obs_tensor):
    state = int(obs_tensor[0] * 100)
    qubit = int(obs_tensor[1] * 100)
    state_binary = format(state, '0' + str(qubit) + 'b')
    return state_binary

# 海灵格距离
def HellingerDistance(p, q):
    import numpy as np
    p = np.array(p)
    q = np.array(q)
    return  np.sqrt(np.sum((np.sqrt(p) - np.sqrt(q))**2)) / np.sqrt(2)

# JS散度
def JensenShannonDivergence(p, q):
    p = np.array(p)
    q = np.array(q)
    M = (p + q)/2
    return 0.5 * np.sum(p*np.log(p/M)) + 0.5 * np.sum(q*np.log(q/M))

# MSE
def get_MSE(p,q):
    p = np.array(p)
    q = np.array(q)
    mse = np.mean((p - q) ** 2)
    return mse    

# 获取某一数组的前百分之25元素的索引返回，如果不足1则返回最大元素的索引
def get_top25_percent_indices(arr):
    n = len(arr)
    k = max(int(0.25 * n), 1)  # 获取前25%的数量，确保至少返回一个索引
    if k >= n:  # 如果前25%的数量大于等于数组长度
        return [np.argmax(arr)]  # 返回最大元素的索引
    else:
        indices = np.argpartition(arr, -k)[-k:]  # 获取前25%值对应的索引
        return indices

# 获取某一数组的前百分之50元素的索引返回，如果不足1则返回最大元素的索引
def get_top50_percent_indices(arr):
    n = len(arr)
    k = max(int(0.5 * n), 1)  # 获取前50%的数量，确保至少返回一个索引
    if k >= n:  # 如果前25%的数量大于等于数组长度
        return [np.argmax(arr)]  # 返回最大元素的索引
    else:
        indices = np.argpartition(arr, -k)[-k:]  # 获取前25%值对应的索引
        return indices


# 获取数组中值不为0的元素的索引
def get_not_0_index(arr):
    indices = [i for i, value in enumerate(arr) if value != 0]
    return indices

# 获取线路中单比特和2比特门的个数,线路深度和宽度
def get_circuit_info(trans_circuit):
    dag = circuit_to_dag(trans_circuit)
    Num_1Q_Gates = 0
    Num_2Q_Gates = 0
    circuit_depth = trans_circuit.depth()
    circuit_weith = trans_circuit.width()
    circuit_info = []
    for node in dag.nodes():
            try:
                if node.qargs:
                    if node.name=="barrier" or node.name=="measure":
                        continue
                    if len(node.qargs)==1:
                        Num_1Q_Gates+=1
                    else:
                        Num_2Q_Gates+=1
            except:
                pass
    # circuit_info.append(Num_1Q_Gates)
    # circuit_info.append(Num_2Q_Gates)
    circuit_info.append(circuit_depth)
    circuit_info.append(circuit_weith)
    return circuit_info

# 获取num在arr中从小到大排序之后的位次
def find_sorted_position(arr, num):
    sorted_arr = sorted(arr)
    return sorted_arr.index(num) + 1


# 求得字典的key的合集之后根据新的key的集合更新字典并返回
def update_noisy_dict(dict_1,dict_2,dict_3):
    # 求取所有字典的key的合集
    dicts = [dict_1,dict_2,dict_3]
    keys = set().union(*dicts)
    
    # 更新每个字典
    for d in dicts:
        for key in keys:
            if key not in d:
                d[key] = 0
    dict_1 = dicts[0]
    dict_2 = dicts[1]
    dict_3 = dicts[2]
    return dict_1,dict_2,dict_3

# 获取一个量子线路的门集统计信息
def get_gate_info(qc):
    gate_info = {}  # 用于存储门的信息
    qubits_used = set()  # 用于存储使用过的比特索引的集合
    two_qubits_gate = ['cx', 'cz', 'cy']
    for instruction in qc._data:
        op_name = instruction.operation.name
        if op_name not in gate_info:
            gate_info[op_name] = {}  # 初始化门的信息字典
        qubits = instruction.qubits
        if op_name in ['measure', 'barrier']:
            continue  # 跳过测量和 barrier 操作
        for qubit in qubits:
            qubits_used.add(qubit.index)  # 将比特索引添加到集合中
        if op_name in two_qubits_gate:
            control, target = qubits
            pair = (control.index, target.index)  # 控制-目标比特对
            if pair not in gate_info[op_name]:
                gate_info[op_name][pair] = 1  # 若该控制-目标比特对尚未记录，则初始化为1
            else:
                gate_info[op_name][pair] += 1  # 若已记录过该控制-目标比特对，则次数加1
        else:
            qubit_index = qubits[0].index  # 获取操作作用的比特索引
            if qubit_index not in gate_info[op_name]:
                gate_info[op_name][qubit_index] = 1  # 若该比特索引尚未记录，则初始化为1
            else:
                gate_info[op_name][qubit_index] += 1  # 若已记录过该比特索引，则次数加1
    qubits_used = sorted(list(qubits_used))  # 将集合转换为排序后的列表
    gate_info['qubits_num'] = qubits_used  # 添加比特数组到返回的字典中
    return gate_info

# 计算一个量子线路的error_info
def get_error_info(properties, gate_info):
    error_info = {}  # 存储门的错误信息
    # 初始化
    for gate in properties['gates_set']:
        error_info[gate] = 0
    for gate in gate_info:
        if gate in properties['gates_set']:
            error_sum = 0
            for qubit, count in gate_info[gate].items():
                if isinstance(qubit, tuple):  # 对于两比特门
                    control, target = qubit
                    gate_name = gate + f'_{control}_{target}'  # 按照惯例拼接门的名称
                    gate_error = properties['gate_props'][gate_name]['gate_error']  # 获取门的错误率
                    error_sum += count * gate_error
                else:  # 对于单比特门
                    gate_name = gate + f'_{qubit}'
                    gate_error = properties['gate_props'][gate_name]['gate_error']  # 获取门的错误率
                    error_sum += count * gate_error
            error_info[gate] = error_sum
    
    # 计算使用过的qubits的readout_error的和
    readout_error_sum = sum(properties['qubits_props'][qubit]['readout_error'] for qubit in gate_info['qubits_num'])

    error_info['readout_error'] = readout_error_sum
    error_info.pop('id')
    error_info.pop('reset')
    error_info.pop('rz')
    
    
    return error_info


# 获取天衍的error_info
def get_tianyan_error(properties,gate_info):
    total_statistic = 0.0
    try:
        rz = gate_info['rz']
        for rz_gate,gate_num in rz.items():
            total_statistic +=properties['singleQubit_error'][str(rz_gate)]*gate_num
            # print(properties['singleQubit_error'][str(rz_gate)])
    except:
        pass
    try:
        sx = gate_info['sx']
        for sx_gate,gate_num in sx.items():
            total_statistic +=properties['singleQubit_error'][str(sx_gate)]*gate_num
            # print(properties['singleQubit_error'][str(sx_gate)])
    except:
        pass

    try:
        x = gate_info['x']
        for x_gate,gate_num in x.items():
            total_statistic +=properties['singleQubit_error'][str(x_gate)]*2*gate_num
            # print(properties['singleQubit_error'][str(x_gate)])
    except:
        pass

    try:
        cx = gate_info['cx']
        for cx_gate,gate_num in cx.items():
            total_statistic +=properties['CZ_error'][str(cx_gate)]*gate_num
            total_statistic +=properties['singleQubit_error'][[str(cx_gate[1])]]*2*gate_num
            # print(properties['CZ_error'][str(cx_gate)])
    except:
        pass

    try:
        qubits = gate_info['qubits_num']
        for qubit in qubits:
            total_statistic +=properties['singleQubit_readoutError'][str(qubit)]
            # print(properties['singleQubit_readoutError'][str(qubit)])
    except:
        pass
        # print(total_statistic)
    return total_statistic*0.01

In [11]:
import qiskit.circuit.random
import torch, random
from torch.utils.data import Dataset, DataLoader, TensorDataset
from torch.optim.lr_scheduler import ReduceLROnPlateau
import torch.nn as nn

import numpy as np
import json, os, pickle
from tqdm import tqdm
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from qiskit import QuantumCircuit
from scipy import stats

def binary_to_decimal(binary_string):
    decimal_number = int(binary_string, 2)
    return decimal_number

def count_lenth(list):
    total_elements = 0
    for dict_i in list:
        total_elements += len(dict_i)
    return total_elements

def count_gates_by_rotation_angle(circuit, bin_size):
    angles = []
    for instr, qargs, cargs in circuit.data:
        if instr.name in ['rx', 'ry', 'rz'] and len(qargs) == 1:
            angles += [float(instr.params[0])]
    bin_edges = np.arange(-2 * np.pi, 2 * np.pi + bin_size, bin_size)
    counts, _ = np.histogram(angles, bins=bin_edges)
    bin_labels = [f"{left:.2f} to {right:.2f}" for left, right in zip(bin_edges[:-1], bin_edges[1:])]
    angle_bins = {label: count for label, count in zip(bin_labels, counts)}
    return list(angle_bins.values())


def recursive_dict_loop(my_dict, parent_key=None, out=None, target_key1=None, target_key2=None):
    if out is None: out = []

    for key, val in my_dict.items():
        if isinstance(val, dict):
            recursive_dict_loop(val, key, out, target_key1, target_key2)
        else:
            if parent_key and target_key1 in str(parent_key) and key == target_key2:
                out += [val]
    return out or 0.

def encode_data(circuits,backend_name_list,properties_dict, ideal_vals_list, noisy_vals_list_1,noisy_vals_list_2,noisy_vals_list_3):
    # circuits是量子线路，properties是后端属性，ideal_exp_vals和noisy_exp_vals分别是理想和含噪声的状态解(其数据类型是数组，数组元素是字典，每一个字典是一条线路的状态解)

    # gates_set = sorted(['reset', 'rz', 'sx', 'x', 'id', 'cx'])     # must sort!
    # noisy_gates_set = sorted([ 'sx', 'x',  'cx', 'rz'])
    gates_set = sorted([ 'sx', 'x', 'cx','rz'])
    # backend_code = {'fake_guadalupe':1,'fake_montreal':2,'fake_cairo':3,'fake_mumbai':4,'fake_sydney':5,'fake_toronto':6}
    # FakeGuadalupe,FakeMontreal,FakeCairo,FakeMumbai,FakeSydney,FakeToronto

    # vec = [np.mean(recursive_dict_loop(properties, out=[], target_key1='cx', target_key2='gate_error'))]
    # vec += [np.mean(recursive_dict_loop(properties, out=[], target_key1='id', target_key2='gate_error'))]
    # vec += [np.mean(recursive_dict_loop(properties, out=[], target_key1='sx', target_key2='gate_error'))]
    # vec += [np.mean(recursive_dict_loop(properties, out=[], target_key1='x', target_key2='gate_error'))]
    # vec += [np.mean(recursive_dict_loop(properties, out=[], target_key1='rz', target_key2='gate_error'))]
    # vec += [np.mean(recursive_dict_loop(properties, out=[], target_key1='', target_key2='readout_error'))]
    # vec += [np.mean(recursive_dict_loop(properties, out=[], target_key1='', target_key2='t1'))]
    # vec += [np.mean(recursive_dict_loop(properties, out=[], target_key1='', target_key2='t2'))]
    # vec = torch.tensor(vec) * 100  # put it in the same order of magnitude as the expectation values



    bin_size = 0.5 * np.pi
    num_angle_bins = int(np.ceil(4 * np.pi / bin_size))

    # 首先更新含噪声的3个数组字典
    for i,circ in enumerate(circuits):
        noisy_vals_list_1[i],noisy_vals_list_2[i],noisy_vals_list_3[i] = update_noisy_dict(noisy_vals_list_1[i],noisy_vals_list_2[i],noisy_vals_list_3[i])
    # 为每一个含噪声的状态解都要创建一行数据
    X_lenth = count_lenth(noisy_vals_list_1)

    X = torch.zeros([X_lenth, 1  + len(gates_set) + num_angle_bins +2 + 3 + 1 + 2])

    error_slice = slice(0, 1)
    # backend_code_slice = slice(1,1+1)
    gate_counts_slice = slice(1, 1+len(gates_set))
    angle_bins_slice = slice(1+len(gates_set), 1+len(gates_set)+num_angle_bins)
    # exp_val_slice = slice(len(vec)+len(gates_set)+num_angle_bins, len(vec)+len(gates_set)+num_angle_bins+num_qubits)
    # 线路状态信息
    gate_info_slice = slice(1+len(gates_set)+num_angle_bins,1+len(gates_set)+num_angle_bins+2)
    # 当前状态的概率值
    exp_val_slice = slice(1+len(gates_set)+num_angle_bins+2, 1+len(gates_set)+num_angle_bins+2+3)
    # 当前状态的概率值的波动程度，标准差
    std_slice = slice(1+len(gates_set)+num_angle_bins+2+3,1+len(gates_set)+num_angle_bins+2+3+1)
    # 二进制的状态编码转换为一个十进制数和对应的量子比特数
    state_slice = slice(1+len(gates_set)+num_angle_bins+2+3+1,1+len(gates_set)+num_angle_bins+2+3+1+2)
    # meas_basis_slice = slice(len(vec)+len(gates_set)+num_angle_bins+1+2, len(X[0]))
    # 当前线路对应的运行后端的编码

    x_count = 0
    ideal_vals = []
    for i, circ in enumerate(circuits):
        gate_counts_all = circ.count_ops()
        # 同一条线路的这个数据是相等的，所以这里统一赋值
        backend_name = backend_name_list[i]
        # backend_code_i = backend_code[backend_name]
        properties = properties_dict[backend_name]
        circuit_info = get_circuit_info(circ)
        gate_counts = count_gates_by_rotation_angle(circ, bin_size)
        noisy_vals_1 = noisy_vals_list_1[i]
        noisy_vals_2 = noisy_vals_list_2[i]
        noisy_vals_3 = noisy_vals_list_3[i]
        gate_info = get_gate_info(circ)
        error_info = get_tianyan_error(properties,gate_info)
        # error_info = sum(np.array(list(error_info.values())))
        for noisy_state in noisy_vals_1.keys():
            obs = []
            state = binary_to_decimal(noisy_state)*0.01
            qubits = len(noisy_state)*0.01
            obs.append(state)
            obs.append(qubits)
            noisy_vals = [noisy_vals_1[noisy_state],noisy_vals_2[noisy_state],noisy_vals_3[noisy_state]]
            std_noisy =  stats.tstd(noisy_vals)
            # 当前后端的编号
            # X[x_count,backend_code_slice] = torch.tensor(backend_code_i)
            # 3次含噪声的值的标准差
            X[x_count,std_slice] = torch.tensor(std_noisy)
            # 门错误统计信息
            X[x_count,error_slice] = torch.tensor(error_info)
            # 线路信息编码，4维
            X[x_count,gate_info_slice] = torch.tensor(circuit_info)*0.01
            # 状态编码，二维
            X[x_count, state_slice] = torch.tensor(obs, dtype=torch.float)
            # 对应状态解的编码，3维,因为线路在含噪声的情况下运行了3次
            X[x_count, exp_val_slice] = torch.tensor(noisy_vals,dtype=torch.float)
            # 将对应的理想解按X的噪声解顺序加到y中
            if noisy_state in ideal_vals_list[i].keys():
                ideal_vals.append(ideal_vals_list[i][noisy_state])
            else:
                ideal_vals.append(0)
            X[x_count, gate_counts_slice] = torch.tensor([gate_counts_all.get(key, 0) for key in gates_set]) * 0.01  # put it in the same order of magnitude as the expectation values
            X[x_count, angle_bins_slice] = torch.tensor(gate_counts) * 0.01  # put it in the same order of magnitude as the expectation values
            x_count += 1
    
    y = torch.tensor(ideal_vals, dtype=torch.float32)
    
    return X, y


In [12]:
import glob
def get_test_loaders(test_circuits,backend_name_list, test_ideal_vals_list, test_noisy_vals_list_1,test_noisy_vals_list_2,test_noisy_vals_list_3,properties_dict,BATCH_SIZE = 32):
    # 这个函数为每一条线路都生成一个test_loader，这样在测试的时候可以把同一条线路的结果放在一起进行处理
    Test_loaders = []
    X_test = []
    y_test = []
    for i,single_circuit in enumerate(test_circuits):
        single_circuit = [single_circuit]
        single_ideal = [test_ideal_vals_list[i]]
        single_noisy_1 = [test_noisy_vals_list_1[i]]
        single_noisy_2 = [test_noisy_vals_list_2[i]]
        single_noisy_3 = [test_noisy_vals_list_3[i]]
        backend_name = [backend_name_list[i]]
        # *******************
        # x_test_i,y_test_i = encode_data(single_circuit,properties,single_ideal,single_noisy_1,single_noisy_2,single_noisy_3)
        x_test_i, y_test_i = encode_data(single_circuit,backend_name,properties_dict,single_ideal,single_noisy_1,single_noisy_2,single_noisy_3)
        test_dataset_i = TensorDataset(torch.Tensor(x_test_i), torch.Tensor(y_test_i))
        test_loader_i = DataLoader(test_dataset_i, batch_size=BATCH_SIZE*1000, shuffle=False)
        x_test_i = pd.DataFrame(x_test_i)
        y_test_i = pd.DataFrame(y_test_i)
        Test_loaders.append(test_loader_i)
        X_test.append(x_test_i)
        y_test.append(y_test_i)
    return Test_loaders,X_test,y_test
            
def get_test_data(test_paths):
    # 参数就是一个路径的数组
    # test_paths = glob.glob(f"{test_path}/dataset_**.pk")
    test_circuits, test_ideal_vals_list, test_noisy_vals_list_1,test_noisy_vals_list_2,test_noisy_vals_list_3,backend_name_list = load_circuits(test_paths,f_ext = '.pk')
    shots = sum(test_ideal_vals_list[0].values())
    test_ideal_vals_list = [{k: v / shots for k, v in d.items()} for d in test_ideal_vals_list]
    # test_noisy_vals_list_1 = [{k: v / shots for k, v in d.items()} for d in test_noisy_vals_list_1]
    # test_noisy_vals_list_2 = [{k: v / shots for k, v in d.items()} for d in test_noisy_vals_list_2]
    # test_noisy_vals_list_3 = [{k: v / shots for k, v in d.items()} for d in test_noisy_vals_list_3]
    test_loader_list,X_test_list, y_test_list = get_test_loaders(test_circuits, backend_name_list,test_ideal_vals_list, test_noisy_vals_list_1,test_noisy_vals_list_2,test_noisy_vals_list_3,properties_dict)
    return test_loader_list,X_test_list, y_test_list

def get_train_data(train_path):
    BATCH_SIZE = 32
    train_paths = glob.glob(f"{train_path}/**.pk")
    train_circuits, train_ideal_vals_list, train_noisy_vals_list_1,train_noisy_vals_list_2,train_noisy_vals_list_3,backend_name_list = load_circuits(train_paths,f_ext = '.pk')
    # 转换为概率值
    shots = sum(train_ideal_vals_list[0].values())
    train_ideal_vals_list = [{k: v / shots for k, v in d.items()} for d in train_ideal_vals_list]
    # train_noisy_vals_list_1 = [{k: v / shots for k, v in d.items()} for d in train_noisy_vals_list_1]
    # train_noisy_vals_list_2 = [{k: v / shots for k, v in d.items()} for d in train_noisy_vals_list_2]
    # train_noisy_vals_list_3 = [{k: v / shots for k, v in d.items()} for d in train_noisy_vals_list_3]
    ######################################
    X_train, y_train = encode_data(train_circuits,backend_name_list,properties_dict, train_ideal_vals_list, train_noisy_vals_list_1,train_noisy_vals_list_2,train_noisy_vals_list_3)
    train_dataset = TensorDataset(torch.Tensor(X_train), torch.Tensor(y_train))
    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    X_train = pd.DataFrame(X_train)#.iloc[:, -4:]
    y_train = pd.DataFrame(y_train)
    return train_loader,X_train,y_train

In [13]:
train_path = './multi_backend_model_data/tianyan_train_data'
train_loader,X_train,y_train = get_train_data(train_path)

In [14]:
save_path = './test_tianyan.csv'
X_train.to_csv(save_path,  index=False,  mode='w',  header=True)

In [120]:
# 按比特数加载数据
# 按比特数加载测试数据
test_path = './multi_backend_model_data/tianyan_big_test_data'
test_paths = glob.glob(f"{test_path}/**.pk")
# 将文件列表按照
test_paths = sorted(test_paths,key=lambda x: int(x.split('_')[-2]))
# print(test_paths)
# 1种后端
test_paths_split_by_qubits = chunk_list(test_paths,1)
print(test_paths_split_by_qubits)
# print(test_paths_split_by_qubits)
test_loader_lists = []
X_test_lists = []
y_test_lists = []
# suanfa_name = []
for qubits_paths in test_paths_split_by_qubits:
    # print(qubits_paths)
    # 获取算法名字
    # base_name =  os.path.basename(paths)
    # suanfa_name.append(os.path.splitext(base_name)[0])
    test_loader_list,X_test_list, y_test_list = get_test_data(qubits_paths)
    test_loader_lists.append(test_loader_list)
    X_test_lists.append(X_test_list)
    y_test_lists.append(y_test_list)

[['./multi_backend_model_data/tianyan_big_test_data\\tianyan_25_qubit.pk']]


In [16]:
# 保存模型
def save_model(rfr,model_name):
    save_path = f"./result"
    model_path = f"{save_path}/{model_name}"
    result_path = f"{model_path}/result"
    loss_path = model_path
    if not os.path.exists(model_path):
        os.mkdir(model_path)
    if not os.path.exists(result_path):
        os.mkdir(result_path)
    with open(f"{model_path}/{model_name}.pk", 'wb') as f:
        pickle.dump(rfr, f)
# 加载模型
def loade_model(model_name):
    save_path = f"./result"
    model_path = f"{save_path}/{model_name}"
    result_path = f"{model_path}/result"
    with open(f"{model_path}/{model_name}.pk", 'rb') as f:
        rfr = pickle.load(f)
    return rfr,result_path

In [17]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
fix_random_seed(0)
rfr = RandomForestRegressor(n_estimators=300)
rfr.fit(X_train, y_train)

random seed fixed to 0


c:\Users\30806\.conda\envs\test\lib\site-packages\sklearn\base.py:1152: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


RandomForestRegressor(n_estimators=300)

In [117]:
print(X_test_list[0])

           0     1     2     3     4    5    6     7     8     9   ...   11  \
0      1.3979  0.08  0.13  0.09  0.07  0.0  0.0  0.03  0.05  0.02  ...  0.0   
1      1.3979  0.08  0.13  0.09  0.07  0.0  0.0  0.03  0.05  0.02  ...  0.0   
2      1.3979  0.08  0.13  0.09  0.07  0.0  0.0  0.03  0.05  0.02  ...  0.0   
3      1.3979  0.08  0.13  0.09  0.07  0.0  0.0  0.03  0.05  0.02  ...  0.0   
4      1.3979  0.08  0.13  0.09  0.07  0.0  0.0  0.03  0.05  0.02  ...  0.0   
...       ...   ...   ...   ...   ...  ...  ...   ...   ...   ...  ...  ...   
47729  1.3979  0.08  0.13  0.09  0.07  0.0  0.0  0.03  0.05  0.02  ...  0.0   
47730  1.3979  0.08  0.13  0.09  0.07  0.0  0.0  0.03  0.05  0.02  ...  0.0   
47731  1.3979  0.08  0.13  0.09  0.07  0.0  0.0  0.03  0.05  0.02  ...  0.0   
47732  1.3979  0.08  0.13  0.09  0.07  0.0  0.0  0.03  0.05  0.02  ...  0.0   
47733  1.3979  0.08  0.13  0.09  0.07  0.0  0.0  0.03  0.05  0.02  ...  0.0   

        12    13    14       15       16       17  

In [114]:
for test_X,test_y in test_loader_list[0]:
    print(test_X[:10])
    with open(f"./result/tianyan_model/tianyan_model.pk", 'rb') as f:
        model = pickle.load(f)
    out = model.predict(test_X[:10])
    print(out)
    # print(out)
    # state_list = []
    # ideal_list = []
    # noisy_list = []
    # mitigated_list = []
    # result = {}
    # out = model.predict(test_X[:, :])
    
    # for obs, ideal, noisy, mitigated in zip(
    #     test_X[:,-2:],
    #     test_y.tolist(),
    #     test_X[:, -6:-3].tolist(),
    #     out.tolist()
    # ):
    #     state_list.append(get_state(obs))
    #     ideal_list.append(ideal)
    #     noisy_list.append(np.mean(np.array(noisy)))
    #     mitigated_list.append(mitigated)

tensor([[2.1390e-01, 1.0000e-02, 2.0000e-02, 1.0000e-02, 2.0000e-02, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 1.0000e-02, 1.0000e-02, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 4.0000e-02, 7.1000e-01, 3.7880e-01, 3.7360e-01, 3.8070e-01,
         3.6756e-03, 1.9000e-01, 5.0000e-02],
        [2.1390e-01, 1.0000e-02, 2.0000e-02, 1.0000e-02, 2.0000e-02, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 1.0000e-02, 1.0000e-02, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 4.0000e-02, 7.1000e-01, 4.1310e-01, 4.1370e-01, 4.1870e-01,
         3.0746e-03, 3.0000e-02, 5.0000e-02],
        [2.1390e-01, 1.0000e-02, 2.0000e-02, 1.0000e-02, 2.0000e-02, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 1.0000e-02, 1.0000e-02, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 4.0000e-02, 7.1000e-01, 3.0100e-02, 2.9700e-02, 3.1100e-02,
         7.2111e-04, 1.1000e-01, 5.0000e-02],
        [2.1390e-01, 1.0000e-02, 2.0000e-02, 1.0000e-02, 2.0000e-02, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 1.0000e-02, 1.0000e

In [118]:
for test_X,test_y in test_loader_list[0]:
    print(test_X[:10])
    with open(f"./result/tianyan_model/tianyan_model.pk", 'rb') as f:
        model = pickle.load(f)
    out = model.predict(test_X[:10])
    print(out)
    # print(out)
    # state_list = []
    # ideal_list = []
    # noisy_list = []
    # mitigated_list = []
    # result = {}
    # out = model.predict(test_X[:, :])
    
    # for obs, ideal, noisy, mitigated in zip(
    #     test_X[:,-2:],
    #     test_y.tolist(),
    #     test_X[:, -6:-3].tolist(),
    #     out.tolist()
    # ):
    #     state_list.append(get_state(obs))
    #     ideal_list.append(ideal)
    #     noisy_list.append(np.mean(np.array(noisy)))
    #     mitigated_list.append(mitigated)

tensor([[1.3979e+00, 8.0000e-02, 1.3000e-01, 9.0000e-02, 7.0000e-02, 0.0000e+00,
         0.0000e+00, 3.0000e-02, 5.0000e-02, 2.0000e-02, 3.0000e-02, 0.0000e+00,
         0.0000e+00, 5.0000e-02, 9.1000e-01, 3.7000e-04, 3.8000e-04, 2.9000e-04,
         4.9329e-05, 9.6820e+01, 2.5000e-01],
        [1.3979e+00, 8.0000e-02, 1.3000e-01, 9.0000e-02, 7.0000e-02, 0.0000e+00,
         0.0000e+00, 3.0000e-02, 5.0000e-02, 2.0000e-02, 3.0000e-02, 0.0000e+00,
         0.0000e+00, 5.0000e-02, 9.1000e-01, 4.0000e-04, 4.5000e-04, 3.9000e-04,
         3.2146e-05, 2.7252e+03, 2.5000e-01],
        [1.3979e+00, 8.0000e-02, 1.3000e-01, 9.0000e-02, 7.0000e-02, 0.0000e+00,
         0.0000e+00, 3.0000e-02, 5.0000e-02, 2.0000e-02, 3.0000e-02, 0.0000e+00,
         0.0000e+00, 5.0000e-02, 9.1000e-01, 2.5400e-03, 2.3400e-03, 2.5300e-03,
         1.1269e-04, 3.2302e+03, 2.5000e-01],
        [1.3979e+00, 8.0000e-02, 1.3000e-01, 9.0000e-02, 7.0000e-02, 0.0000e+00,
         0.0000e+00, 3.0000e-02, 5.0000e-02, 2.0000e

In [18]:
model_name = 'tianyan_model'
save_model(rfr,model_name)

In [121]:
def get_model_mited_test(model_name,test_loader_lists,save:bool):
    HLD_list = []
    count_right_list = []
    model,result_path = loade_model(model_name)
    for index_i_s,test_loader_list in enumerate(test_loader_lists):
        qubits = index_i_s + 2
        HLD_mited_this_qubits = []
        # HLD_noisy_this_qubits = []
        count_right = 0
        count_all = 0
        for index_i,test_loader in enumerate(test_loader_list):
            for test_X,test_y in test_loader:
                state_list = []
                ideal_list = []
                noisy_list = []
                mitigated_list = []
                result = {}
                out = model.predict(test_X[:, :])
                # print(type(out))
                for obs, ideal, noisy, mitigated in zip(
                    test_X[:,-2:],
                    test_y.tolist(),
                    test_X[:, -6:-3].tolist(),
                    out.tolist()
                ):
                    state_list.append(get_state(obs))
                    ideal_list.append(ideal)
                    noisy_list.append(np.mean(np.array(noisy)))
                    mitigated_list.append(mitigated)
                    
                mitigated_list = np.array(mitigated_list)
                # 先使得最小值为小于0的为0，再归一化
                mitigated_list = np.clip(mitigated_list, a_min=0, a_max=None)
                mitigated_list = mitigated_list/sum(mitigated_list)
                for ideal,noisy,mitigated in zip(ideal_list,noisy_list,mitigated_list):
                    if(abs(ideal-mitigated)<abs(ideal-noisy)):
                        count_right +=1
                    count_all += 1
                HLD_mited = HellingerDistance(ideal_list,mitigated_list)
                print(HLD_mited)
                # HLD_noisy = HellingerDistance(ideal_list,noisy_list)
                HLD_mited_this_qubits.append(HLD_mited)
                # HLD_noisy_this_qubits.append(HLD_noisy)
                if (save):
                    result['state'] = state_list
                    result['ideal'] = ideal_list
                    result['noisy'] = noisy_list
                    result['mitigated'] = mitigated_list
                    df = pd.DataFrame(result)
                    file_name = f"{result_path}/{qubits}_qubits_{index_i}.xlsx"
                    df.to_excel(file_name, index=False)
                    print(f"结果已保存到 {file_name} 文件中")
        mean_HLD_mited_this_qubits = np.mean(np.array(HLD_mited_this_qubits))
        mitigated_right = count_right/count_all
        HLD_list.append(mean_HLD_mited_this_qubits)
        count_right_list.append(mitigated_right)
    return HLD_list,count_right_list
        # mean_HLD_noisy_this_qubits = np.mean(np.array(HLD_noisy_this_qubits))
                    

In [122]:
def get_model_mited_test(model_name,test_loader_lists,save:bool):
    HLD_list = []
    count_right_list = []
    model,result_path = loade_model(model_name)
    for index_i_s,test_loader_list in enumerate(test_loader_lists):
        qubits = index_i_s + 2
        HLD_mited_this_qubits = []
        # HLD_noisy_this_qubits = []
        count_right = 0
        count_all = 0
        for index_i,test_loader in enumerate(test_loader_list):
            for test_X,test_y in test_loader:
                state_list = []
                ideal_list = []
                noisy_list = []
                mitigated_list = []
                result = {}
                out = model.predict(test_X[:, :])
                
                for obs, ideal, noisy, mitigated in zip(
                    test_X[:,-2:],
                    test_y.tolist(),
                    test_X[:, -6:-3].tolist(),
                    out.tolist()
                ):
                    state_list.append(get_state(obs))
                    ideal_list.append(ideal)
                    noisy_list.append(np.mean(np.array(noisy)))
                    mitigated_list.append(mitigated)
                    
                mitigated_list = np.array(mitigated_list)
                # 先使得最小值为小于0的为0，再归一化
                mitigated_list = np.clip(mitigated_list, a_min=0, a_max=None)
                mitigated_list = mitigated_list/sum(mitigated_list)
                for ideal,noisy,mitigated in zip(ideal_list,noisy_list,mitigated_list):
                    if(abs(ideal-mitigated)<abs(ideal-noisy)):
                        count_right +=1
                    count_all += 1
                HLD_mited = HellingerDistance(ideal_list,mitigated_list)
                print(HLD_mited)
                # HLD_noisy = HellingerDistance(ideal_list,noisy_list)
                HLD_mited_this_qubits.append(HLD_mited)
                # HLD_noisy_this_qubits.append(HLD_noisy)
                if (save):
                    result['state'] = state_list
                    result['ideal'] = ideal_list
                    result['noisy'] = noisy_list
                    result['mitigated'] = mitigated_list
                    df = pd.DataFrame(result)
                    file_name = f"{result_path}/{qubits}_qubits_{index_i}.xlsx"
                    df.to_excel(file_name, index=False)
                    print(f"结果已保存到 {file_name} 文件中")
        mean_HLD_mited_this_qubits = np.mean(np.array(HLD_mited_this_qubits))
        mitigated_right = count_right/count_all
        HLD_list.append(mean_HLD_mited_this_qubits)
        count_right_list.append(mitigated_right)
    return HLD_list,count_right_list
        # mean_HLD_noisy_this_qubits = np.mean(np.array(HLD_noisy_this_qubits))
                    

In [ ]:
def get_model_mited_test_1(model_name,test_loader_lists,save:bool):
    HLD_list = []
    count_right_list = []
    model,result_path = loade_model(model_name)
    for index_i_s,test_loader_list in enumerate(test_loader_lists):
        qubits = index_i_s + 2
        HLD_mited_this_qubits = []
        # HLD_noisy_this_qubits = []
        count_right = 0
        count_all = 0
        for index_i,test_loader in enumerate(test_loader_list):
            for test_X,test_y in test_loader:
                state_list = []
                ideal_list = []
                noisy_list = []
                mitigated_list = []
                result = {}
                out = model.predict(test_X[:, :])
                
                for obs, ideal, noisy, mitigated in zip(
                    test_X[:,-2:],
                    test_y.tolist(),
                    test_X[:, -6:-3].tolist(),
                    out.tolist()
                ):
                    state_list.append(get_state(obs))
                    ideal_list.append(ideal)
                    noisy_list.append(np.mean(np.array(noisy)))
                    mitigated_list.append(mitigated)
                    
                mitigated_list = np.array(mitigated_list)
                # 先使得最小值为小于0的为0，再归一化
                mitigated_list = np.clip(mitigated_list, a_min=0, a_max=None)
                mitigated_list = mitigated_list/sum(mitigated_list)
                for ideal,noisy,mitigated in zip(ideal_list,noisy_list,mitigated_list):
                    if(abs(ideal-mitigated)<abs(ideal-noisy)):
                        count_right +=1
                    count_all += 1
                HLD_mited = HellingerDistance(ideal_list,mitigated_list)
                print(HLD_mited)
                # HLD_noisy = HellingerDistance(ideal_list,noisy_list)
                HLD_mited_this_qubits.append(HLD_mited)
                # HLD_noisy_this_qubits.append(HLD_noisy)
                if (save):
                    result['state'] = state_list
                    result['ideal'] = ideal_list
                    result['noisy'] = noisy_list
                    result['mitigated'] = mitigated_list
                    df = pd.DataFrame(result)
                    file_name = f"{result_path}/{qubits}_qubits_{index_i}.xlsx"
                    df.to_excel(file_name, index=False)
                    print(f"结果已保存到 {file_name} 文件中")
        mean_HLD_mited_this_qubits = np.mean(np.array(HLD_mited_this_qubits))
        mitigated_right = count_right/count_all
        HLD_list.append(mean_HLD_mited_this_qubits)
        count_right_list.append(mitigated_right)
    return HLD_list,count_right_list
        # mean_HLD_noisy_this_qubits = np.mean(np.array(HLD_noisy_this_qubits))
                    

In [124]:
def get_model_noisy_test(test_loader_lists):
    HLD_list = []
    # count_right_list = []
    for index_i_s,test_loader_list in enumerate(test_loader_lists):
        HLD_noisy_this_qubits = []
        for index_i,test_loader in enumerate(test_loader_list):
            for test_X,test_y in test_loader:
                state_list = []
                ideal_list = []
                noisy_list = []
                for obs, ideal, noisy in zip(
                    test_X[:,-2:],
                    test_y.tolist(),
                    test_X[:, -6:-3].tolist()
                ):
                    state_list.append(get_state(obs))
                    ideal_list.append(ideal)
                    noisy_list.append(np.mean(np.array(noisy)))
                HLD_noisy = HellingerDistance(ideal_list,noisy_list)
                HLD_noisy_this_qubits.append(HLD_noisy)
                print(HLD_noisy)
        mean_HLD_noisy_this_qubits = np.mean(np.array(HLD_noisy_this_qubits))
        HLD_list.append(mean_HLD_noisy_this_qubits)

    return HLD_list

                    

In [125]:
noisy = get_model_noisy_test(test_loader_lists)
print(noisy)

0.6577530747306497
0.7844169297347104
0.6049069049168848
0.5841259437811431
0.6781111785855851
[0.6618628063497946]


In [58]:
print(len(noisy))

12


In [83]:
import numpy as np

# 假设 out_1 和 out_2 是两个 numpy 数组
out_1 = np.array([1, 2, 3])
out_2 = np.array([4, 5, 6])

# 沿着指定的轴将两个数组拼接起来
combined_array = np.concatenate((out_1, out_2), axis=0)

# 如果是二维数组，可以指定沿着行或列进行拼接
# combined_array = np.concatenate((out_1, out_2), axis=1)

print(combined_array)


[1 2 3 4 5 6]


In [126]:
tianyan_HLD_list,count_right_listHLD = get_model_mited_test('tianyan_model',test_loader_lists,save =False)
# 注意，之后整理的时候，要把第二条线路去掉

0.35879320677422943
1.0
0.11466177797976965
0.3908812625534154
0.5323311419808815


In [127]:
print(tianyan_HLD_list)

[0.4793334778576591]


In [61]:
save_HLD('tianyan_error',noisy)
save_HLD('tianyan_mited',tianyan_HLD_list)